In [7]:
include("../src/Julia.jl")

parseMatlabData (generic function with 1 method)

In [8]:
pwd()

"/Users/serbanstan/git/TreePCG/julia/testing"

In [18]:
graphp = "../../graphs/pathRand_1000_u11/"

"../../graphs/pathRand_1000_u11/"

In [19]:
a = readFromFile(graphp * "graph.mtx");

In [20]:
tree = []
tr = readFromFile(graphp * "tree.mtx");

u,v,w = findnz(tr)
for i in 1:length(u)
    w[i] = a[u[i],v[i]]
end
tr = sparse(u,v,w)

push!(tree, tr);

In [21]:
x = readFromFile(graphp * "x.vec");

In [22]:
la = lap(a);
x = x - mean(x)
b = la * x; b = b - mean(b);

In [23]:
numIts = 100;

for i in 1:1
    @time F = treeSolver(tree[i])
    @time F(b);
    
    # a run on standard Float64 data types
    @time myx64,dbg = pcgV(la, b, F, x, maxits=numIts, verbose=false);
    norm(lap(a) * myx64 - b) / norm(b)
    insert!(dbg, 1, "$(numIts) iterations")
    insert!(dbg, 1, "anorm = sqrt((x-truex)' * la * (x-truex)), err2_a = norm(la*x-b) / norm(b), err2_b = norm(x-truex)")
    writeToFile(graphp * "_log_julia_tree$(i)_default.txt", dbg)
    
    # now useing higer precision data types
    for hp in [64,128,256,512,1024]
        set_bigfloat_precision(hp)
        laHP = toHighPrecision(la; precision=hp)
        treeHP = toHighPrecision(tree[i]; precision=hp)
        xHP = toHighPrecision(x; precision=hp);
        bHP = toHighPrecision(b; precision=hp);
        FHP = treeSolver(treeHP)

        @time myxHP,dbg = pcgV(laHP, bHP, FHP, xHP, maxits=numIts, verbose=true);
        insert!(dbg, 1, "$(numIts) iterations")
        insert!(dbg, 1, "anorm = sqrt((x-truex)' * la * (x-truex)), err2_a = norm(la*x-b) / norm(b), err2_b = norm(x-truex)")
        writeToFile(graphp * "_log_julia_tree$(i)_$(hp).txt", dbg)
    end
end

  0.000917 seconds (129 allocations: 374.813 KB)
  0.002823 seconds (17.96 k allocations: 344.375 KB)
  0.139587 seconds (1.82 M allocations: 45.614 MB, 20.61% gc time)
Finished iteration 1 with errors 0.9984896177415457 0.9969856716044583
Finished iteration 11 with errors 0.9008291112944069 0.8354495373490164
Finished iteration 21 with errors 0.7475849141516431 0.6446319911902617
Finished iteration 31 with errors 0.6057508401193235 0.508688288039588
Finished iteration 41 with errors 0.4620872672171531 0.3878219738496883
Finished iteration 51 with errors 0.32488278545464977 0.2648817579762813
Finished iteration 61 with errors 0.22932148662392296 0.18555959271376857
Finished iteration 71 with errors 0.15052555618074615 0.12030735765685298
Finished iteration 81 with errors 0.09420810161325686 0.07704341499232956
Finished iteration 91 with errors 0.072672799279051 0.05908637274172659
Finished iteration 100 with errors 0.04584255742434119 0.03628156972594393
  4.607305 seconds (23.74 M all